SpudJobs are applied first and have discount, SpudMods have hard minimum requirement.
Randomly generate parts by aiming for random target spuds.

In [62]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using MAT
using Serialization

struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
    cost::Int64
end

const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end





function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, utb)
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    end
end

function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    cs = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
        cs[ii] = as[ii].cost
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s, cost = cs)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none, -1)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1], df[i, :cost])
    end
    return as
end



struct NashEquilibrium2p
    w1::Array{Float64}
    w2::Array{Float64}
    v::Float64
end


function payoffs2(set1::Array{Spud}, set2::Array{Spud})::Array{Int64}
    n1 = length(set1)
    n2 = length(set2)
    payoffs = Array{Int64}(undef, (n1, n2))
    for i in 1:n1
        for j in 1:n2
            payoffs[i,j] = eval_battle(set1[i], set2[j])
        end
    end
    return payoffs
end

function ffp2(set1::Array{Spud}, set2::Array{Spud}, nits::Int64)::NashEquilibrium2p
    n1 = length(set1)
    n2 = length(set2)
    payoffs = payoffs2(set1, set2)
    v1 = [1 for i in 1:n1]
    v2 = [1 for i in 1:n2]
    s1 = [sum(payoffs[i, :] .* v2) for i in 1:n1]
    s2 = [sum(payoffs[:, i] .* v1) for i in 1:n2]
    for it in 1:nits
        cands1 = findall(s1 .== maximum(s1))
        i1 = rand(cands1)
        cands2 = findall(s2 .== minimum(s2))
        i2 = rand(cands2)
        v1[i1] += 1
        s2 = s2 .+ payoffs[i1, :]
        v2[i2] += 1
        s1 = s1 .+ payoffs[:, i2]
    end
    w1 = v1./sum(v1)
    w2 = v2./sum(v2)
    val = sum([payoffs[i, j] * w1[i] * w2[j] for i in 1:n1 for j in 1:n2])
    return NashEquilibrium2p(w1, w2, val)
end

function ffp(nash_env, nits)
    n_nash = length(nash_env)
    i_lose = Array{Int}(undef, (n_nash, n_nash))
    n_lose = Array{Int}(undef, n_nash)
    for i in 1:n_nash
        n_lose[i] = 0
        ff = nash_env[i]
        for j in 1:n_nash
            if eval_battle(ff, nash_env[j]) ==-1
                n_lose[i] += 1
                i_lose[i, n_lose[i]] = j
            end
        end
    end
    counts = [0 for i in 1:n_nash]
    wins = [0 for i in 1:n_nash]
    for i in 1:n_nash
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    for iter in 1:nits
        wc = 2 .* wins .+ counts
        ind_winners = findall(wc .== maximum(wc))
        i = rand(ind_winners)
        counts[i] += 1
        for j in 1:n_lose[i]
            i_w = i_lose[i, j]
            wins[i_w]+= 1
        end
    end
    return counts
end

mutable struct BattleView
    round::Int64
    own::Array{Spud}
    owncount::Array{Int64}
    deployed::Array{Bool}
    obs::Array{Spud}
    obscount::Array{Int64}
end

mutable struct AiPlayer
    genteam::Function
    pick::Function
    create::Function
end

n_copy = 3

function initialize_game(team1::Array{Spud}, team2::Array{Spud})::Array{BattleView}
    teams = [team1, team2]
    bvs = Array{BattleView}(undef, 0)
    for team in teams
        bv = BattleView(0, team, [n_copy for i in 1:length(team)], [false for i in 1:length(team)], [], [])
        append!(bvs, [bv])
    end
    return bvs
end

function observe_fighters(game::Array{BattleView}, choices::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].deployed[choices[i]] = true
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                0
            else
                game[j].obs = append!(game[j].obs, [ff])
                game[j].obscount = append!(game[j].obscount, [game[i].owncount[choices[i]]])
            end
        end
    end
    return game
end

function jobify_fighter_counts(game::Array{BattleView}, choices::Array{Int64}, job::Array{Int64})::Array{BattleView}
    for i in 1:2
        j = 3- i
        if choices[i] != 0
            game[i].owncount[choices[i]] = game[i].owncount[choices[i]] + job[i]
            ff = game[i].own[choices[i]]
            if ff in game[j].obs
                ind = findall(x -> x==ff, game[j].obs)[1]
                game[j].obscount[ind] = game[i].owncount[choices[i]]
            else
                0
            end
        end
    end
    return game
end

function play_combat(game::Array{BattleView}, choices::Array{Int64}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    fighters = [game[i].own[choices[i]] for i in 1:2]
    for i in 1:2
        @assert game[i].owncount[choices[i]] >= 0
    end
    if verbose
        print("Round ")
        print(round)
        print(": #A")
        print(choices[1])
        print(" ")
        print(fighters[1])
        print(" vs #B")
        print(choices[2])
        print(" ")
        println(fighters[2])
    end
    outcome = eval_battle(fighters[1], fighters[2])
    if verbose
        if outcome == 0
            println("Tied!")
        else
            if outcome == 1
                print(fighters[1])
            end
            if outcome == -1
                print(fighters[2])
            end
            println(" wins!")
        end
    end
    game = observe_fighters(game, choices)
    if outcome > -1
        game = jobify_fighter_counts(game, choices, [0, -1])
    end
    if outcome < 1
        game = jobify_fighter_counts(game, choices, [-1, 0])
    end
    for i in 1:2
        game[i].round = round
    end
    return game
end

function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    game[player].own = append!(game[player].own, [ff])
    game[player].owncount = append!(game[player].owncount, [n_copy])
    game[player].deployed = append!(game[player].deployed, [false])
    if verbose
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(ff)
    end
    return game
end

function play_is_game_over(game::Array{BattleView})::Bool
    for bv in game
        if sum(bv.owncount) == 0
            return true
        end
    end
    return false
end

function generate_rand_team(env, counts, nteam)
    cc = cumsum(counts)
    team = Array{Spud}(undef, nteam)
    for i in 1:nteam
        tmp = rand(1:sum(counts))
        ind = sum(cc .< tmp)+1
        team[i] = env[ind]
    end
    return team
end

function ai0_pick(a::BattleView)::Int64
    inds = findall(a.owncount .> 0)
    return rand(inds)
end

function ai0_create(a::BattleView)::Spud
    return generate_rand_team(nash_env, counts, 1)[1]
end

function ai1_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    cands = nash_env[ebs .== maximum(ebs)]
    return rand(cands)
end

function ai2_create(a::BattleView)::Spud
    ops = a.obs[a.obscount .> 0]
    ebs = [eval_battle_list(ff, ops) for ff in nash_env]
    if maximum(ebs) == length(ops)
        cands = nash_env[ebs .== maximum(ebs)]
        return rand(cands)
    else
        ebs2 = [eval_battle_list(ff, ops) for ff in lib]
        if maximum(ebs2) > maximum(ebs)
            cands = lib[ebs2 .== maximum(ebs2)]
            return rand_rename(rand(cands))
        else
            cands = nash_env[ebs .== maximum(ebs)]
            return rand(cands)
        end
    end
end


# picks Spud most likely to win
function ai_greedy_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    ind = findall(a.owncount .> 0)[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

# picks from spuds already deployed
function ai_conservative_pick(a::BattleView)::Int64
    own = a.own[a.owncount .> 0 .&& a.deployed]
    owninds = findall(a.owncount .> 0 .&& a.deployed)
    if length(own) == 0
        # pick greedy
        return ai_greedy_pick(a)
    end
    ops = a.obs[a.obscount .> 0]
    res = ffp2(own, ops, 100)
    ind = owninds[rand(findall(res.w1 .== maximum(res.w1)))]
    return ind
end

# picks Spud based on 1-round nash
function ai_myopic_pick(a::BattleView)::Int64
    if length(a.obs[a.obscount .> 0]) == 0
        return rand(findall(a.owncount .> 0))
    end
    res = ffp2(a.own[a.owncount .> 0], a.obs[a.obscount .> 0], 100)
    cc = cumsum(res.w1)
    ind = sum(cc .< rand()) + 1
    return findall(a.owncount .> 0)[ind]
end

creation_rounds = [5, 10, 15]

function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    team1 = ai1.genteam()
    team2 = ai2.genteam()
    game = initialize_game(team1, team2)
    while !play_is_game_over(game)
        # play a round
        choice1 = ai1.pick(game[1])
        choice2 = ai2.pick(game[2])
        game = play_combat(game, [choice1, choice2], verbose)
        if verbose
            for i in 1:2
                println(game[i].owncount)
            end
        end
        if game[1].round in creation_rounds
            for i in 1:2
                ff = ais[i].create(game[i])
                game = play_add_spud(game, i, ff, verbose)
            end
        end
    end
    if sum(game[1].owncount) == 0
        if sum(game[2].owncount) == 0
            return 0
        else
            return -1
        end
    end
    return 1
end

# ai1 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ai2 = AiPlayer(() -> generate_rand_team(nash_env, counts, 5), ai_greedy_pick, ai1_create)
# ais = [ai1, ai2]
# play_game(ais, true)

#@time lib = sample_library(1.01, [999]);

function filter_nondominated(as::Array{Spud})::Array{Spud}
    df = spuds_to_df(as)
    mat = Array{Int64}(undef, (length(as), 6))
    mat[:, 1] = df.h
    mat[:, 2] = df.f
    mat[:, 3] = df.l
    mat[:, 4] = df.p
    mat[:, 5] = df.r
    mat[:, 6] = df.s;
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:6
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

function spud_to_vec(a::Spud)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end

struct SpudBase
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
    cost::Int64
end

function spud_to_vec(a::SpudBase)::Array{Int64}
    return [a.h, a.f, a.l, a.p, a.r, a.s]
end
                                                                                    
struct SpudMod
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
    minh::Int64
    minf::Int64
    minl::Int64
    minp::Int64
    minr::Int64
    mins::Int64
    cost::Int64
end
                                                                                        
                                                                                        

In [2]:
const discount = 20

20

In [3]:
function rename_base(a::SpudBase, s::String)::SpudBase
    return SpudBase(s, a.h,a.f,a.l,a.p,a.r,a.s,a.a1,a.cost)
end

function rename_job(a::SpudMod, s::String)::SpudMod
    return SpudMod(s, a.h,a.f,a.l,a.p,a.r,a.s,a.a1,a.minh,a.minf,a.minl,a.minp,a.minr,a.mins,a.cost)
end

function check_validity(a::Spud)::Bool
    return minimum([a.h,a.f,a.l,a.p,a.r,a.s]) >= MNV
end



check_validity (generic function with 1 method)

In [4]:
function combine_base_job(b::SpudBase, j::SpudMod)::Spud
    v = [b.h,b.f,b.l,b.p,b.r,b.s]
    c = b.cost
    a1 = b.a1
    if a1 == 999
        a1 = j.a1
    end
    w = [j.h,j.f,j.l,j.p,j.r,j.s]
    m = [j.minh,j.minf,j.minl,j.minp,j.minr,j.mins]
    c2 = j.cost
    if sum(v .< m) == 0
        c2 -= discount
    end
    z = v .+ w
    z[z .> MXV] .= MXV
    return Spud(string(j.name, " ", b.name),z[1],z[2],z[3],z[4],z[5],z[6],a1,c+c2)
end

combine_base_job (generic function with 1 method)

In [5]:
function combine_spud_mod(b::Spud, j::SpudMod)::Spud
    v = [b.h,b.f,b.l,b.p,b.r,b.s]
    c = b.cost
    a1 = b.a1
    if a1 == 999
        a1 = j.a1
    end
    w = [j.h,j.f,j.l,j.p,j.r,j.s]
    m = [j.minh,j.minf,j.minl,j.minp,j.minr,j.mins]
    c2 = j.cost
    # require spud to meet minimums
    if sum(v .< m) > 0
        return b
    end
    z = v .+ w
    z[z .> MXV] .= MXV
    return Spud(string(j.name, " ", b.name),z[1],z[2],z[3],z[4],z[5],z[6],a1,c+c2)
end

combine_spud_mod (generic function with 1 method)

#### Load pre-computed scores (from game_designL) and best_cost (from SpudParts2c)

In [110]:
scores = matread("scores_sq300.mat")["scores"];

n_base_parts = 100
n_mods = 100
n_jobs = 100
#min_cost_base = 15
#max_cost_base = 25
cost_base = 15
min_cost_mod = 25
max_cost_mod = 50
#min_cost_job = 35
max_cost_job = 100 - cost_base

min_mod_delt = -2
max_mod_delt = 8

min_job_delt = 0
max_job_delt = 2

min_n_viable = 10
thres_viable = 2


2

In [111]:
best_cost = matread("temp_desLparts_postjobcosts.mat")["best_cost"];

In [112]:
# get all bases of that cost
base_inds = findall(scores .== cost_base);
bases = [
    SpudBase(
        string("b", i), base_inds[i][1], base_inds[i][2], base_inds[i][3], 
        base_inds[i][4], base_inds[i][5], base_inds[i][6], 999, cost_base) for i in 1:length(base_inds)];
length(base_inds)

2804

In [113]:
# get all minimum req vecs
mvs = Array{Int64}(undef, (100000, 6))
mv_i = 0
for i1 in 1:4
    for i2 in (i1+1):5
        for i3 in (i2+1):6
            for j1 in 0:9
                for j2 in 0:9
                    for j3 in 0:9
                        mv_i += 1
                        mvs[mv_i, i1] = j1
                        mvs[mv_i, i2] = j2
                        mvs[mv_i, i3] = j3
                    end
                end
            end
        end
    end
end
mvs = mvs[1:mv_i, :];
mvs = unique(mvs, dims=1);
size(mvs)

(15850, 6)

In [114]:
function get_targets(scores, target_score)
    targets = findall(scores .== target_score)
    targets = reduce(hcat, [getindex.(targets, i) for i in 1:6])
    tgts = [Spud("",v[1],v[2],v[3],v[4],v[5],v[6],ability_none,target_score) for v in eachrow(targets)]
    return tgts
end

function get_inits(best_cost)
    inits_v = findall(best_cost .<= MXS);
    n_inits = size(inits_v)[1]
    inits = Array{Spud}(undef, n_inits)
    for i in 1:n_inits
        inits[i] = Spud(
            "",inits_v[i][1],inits_v[i][2],inits_v[i][3],inits_v[i][4],
            inits_v[i][5],inits_v[i][6],ability_none,best_cost[inits_v[i]])
    end
    return inits
end

function find_mods_for_target(
        scores::Array{Int64}, inits::Array{Spud}, ff::Spud, 
        dist_thres::Int64=1, n_reqs::Int64=3, m_slack::Int64=0)::Dict
    mods = Array{SpudMod}(undef, 1000)
    allcands = Array{Spud}(undef, 1000)
    m_i = 0
    v = spud_to_vec(ff)
    dists = [sum(map(abs, spud_to_vec(ff) .- spud_to_vec(cd))) for cd in inits];
    cands = inits[dists .<= minimum(dists) + dist_thres];
    for cand in cands
        delt = spud_to_vec(ff) .- spud_to_vec(cand)
        m_inds = sortperm(-spud_to_vec(cand))[1:n_reqs]
        m = [0,0,0,0,0,0]
        m[m_inds] = spud_to_vec(cand)[m_inds] .- m_slack
        mm = SpudMod("mod",delt[1],delt[2],delt[3],delt[4],delt[5],delt[6],999,m[1],m[2],m[3],m[4],m[5],m[6],0)
        blockers = [s for s in inits if (sum(spud_to_vec(s) .< m)==0 && minimum(spud_to_vec(s).+delt) > 0)];
        blockers2 = [combine_spud_mod(b, mm) for b in blockers];
        cs = [b.cost for b in blockers];
        c2s = [scores[b.h,b.f,b.l,b.p,b.r,b.s] for b in blockers2];
        maxdiff = maximum(c2s .- cs)        
        if maxdiff + cand.cost <= MXS
            mm = SpudMod("mod",delt[1],delt[2],delt[3],delt[4],delt[5],delt[6],999,m[1],m[2],m[3],m[4],m[5],m[6],maxdiff)
            m_i += 1
            mods[m_i] = mm
            allcands[m_i] = cand
        end
    end
    mods = mods[1:m_i]
    allcands = allcands[1:m_i]
    return Dict("mods" => mods, "cands" => allcands)
end
                
function get_bjms(bases::Array{SpudBase}, jobs::Array{SpudMod}, mods::Array{SpudMod})::Array{Spud}
    bajos = unique([combine_base_job(b, j) for b in bases for j in jobs])
    append!(bajos, unique([combine_spud_mod(s, m) for s in bajos for m in mods]))
    bajos = [b for b in bajos if b.cost <= MXS]
    return bajos
end

function filter_nondominated_recursive(as::Array{Spud})::Array{Spud}
    if length(as) < 1001
        return filter_nondominated(as)
    end
    n = size(as)[1]
    n1 = div(n, 2)
    as1 = as[1:n1]
    as2 = as[(n1+1):n]
    ans1 = filter_nondominated_recursive(as1)
    ans2 = filter_nondominated_recursive(as2)
    return append!(ans1, ans2)
end

get_targets (generic function with 1 method)

In [117]:
function find_jobs_and_bases_for_target(
        mvs::Array{Int64}, bases::Array{SpudBase}, target::Spud, 
        max_job_delt::Int64 = 2,
        min_n_viable::Int64 = 10, thres_viable::Int64 = 2)::Dict
    cost_base = bases[1].cost
    base_vs = [spud_to_vec(b) for b in bases]
    base_vs = hcat(base_vs...)
    js = Array{SpudMod}(undef, 10000)
    cands = Array{SpudBase}(undef, 10000)
    viables = Array{Int64}(undef, 10000)
    j_i = 0
    # get bases that can yield target
    bs = [b for b in bases if sum(spud_to_vec(b) .> spud_to_vec(target)) == 0]
    bs = [b for b in bs if maximum(spud_to_vec(target) .- spud_to_vec(b)) <= max_job_delt]
    # iterate over possible bases
    for b in bs
        #println(b)
        v = spud_to_vec(target) .- spud_to_vec(b)
        m = [0,0,0,0,0,0]
        jj = SpudMod("job",v[1],v[2],v[3],v[4],v[5],v[6],ability_none,m[1],m[2],m[3],m[4],m[5],m[6],0)
        b2s = [combine_base_job(b, jj) for b in bases];
        c2s = [scores[b.h,b.f,b.l,b.p,b.r,b.s] for b in b2s];
        n_viable = Array{Int64}(undef, size(mvs)[1])
        c2_max = Array{Int64}(undef, size(mvs)[1])
        #jcost is undiscounted cost
        jcost = maximum(c2s) - cost_base
        #println(jcost)
        # does jcost hit target cost?
        if cost_base + jcost <= target.cost
            j_i += 1
            cands[j_i] = b
            js[j_i] = SpudMod("job",v[1],v[2],v[3],v[4],v[5],v[6],ability_none,0,0,0,0,0,0,jcost+discount)
            viables[j_i] = sum(c2s .>= cost_base .+ jcost - thres_viable)
        end
        # besides the vanilla job with no minimal req, consider alternative jobs that have requirements
        for i in 1:size(mvs)[1]
            m = mvs[i, :]
            if (sum(spud_to_vec(b) .< m) == 0)
                #println(m)
                filt = 
                    base_vs[1,:] .>= m[1] .&& base_vs[2,:] .>= m[2] .&& base_vs[3,:] .>= m[3] .&&
                    base_vs[4,:] .>= m[4] .&& base_vs[5,:] .>= m[5] .&& base_vs[6,:] .>= m[6]
                c2_max[i] = maximum(c2s[filt]) - cost_base
                # ensure that the undiscounted cost of the alternative-req job is not below the vanilla job
                if c2_max[i] + discount <= jcost
                    c2_max[i] = jcost - discount
                end
                # spuds are viable if their discounted cost is within thres of their value from scores[]
                n_viable[i] = sum(c2s[filt] .>= cost_base .+ c2_max[i] - thres_viable)
                # ensure that the undiscounted cost of the alternative-req job is below the maximum job cost
                cond2 = c2_max[i]+discount <= max_cost_job
                # ensure that enough efficient spuds are made possible to make the job worthy of inclusion
                cond3 = n_viable[i] .>= min_n_viable
                # ensures that discounted cost of job hits target
                cond4 = c2_max[i] + cost_base <= target.cost
                if cond2 && cond3 && cond4
                    j_i += 1
                    cands[j_i] = b
                    js[j_i] = SpudMod(
                        "job",v[1],v[2],v[3],v[4],v[5],v[6],ability_none,
                        m[1],m[2],m[3],m[4],m[5],m[6],c2_max[i]+discount)
                    viables[j_i] = n_viable[i]
                end
            end
        end
    end
    js = js[1:j_i]
    cands = cands[1:j_i]
    viables = viables[1:j_i]
    return Dict("jobs" => js, "cands" => cands, "n_viable" => viables)
end

find_jobs_and_bases_for_target (generic function with 4 methods)

#### Generate large set of parts

In [132]:
bases1 = Array{SpudBase}(undef, 10000)
jobs1 = Array{SpudMod}(undef, 10000)
mods1 = Array{SpudMod}(undef, 10000)
b_i = 0
j_i = 0
m_i = 0

0

In [ ]:
inits = get_inits(best_cost);
targets = get_targets(scores, 90);
for i in 1:100
    targ = rand(targets)
    println(targ)
    mr = find_mods_for_target(scores, inits, targ, 1, 2, 1)
    if length(mr["cands"]) > 0
        ind = sortperm([c.cost for c in mr["cands"]])[1]
        cand = mr["cands"][ind]
        m_i += 1
        mods1[m_i] = mr["mods"][ind]
        println(mods1[m_i])
        # find base and job for cand
        rjb = find_jobs_and_bases_for_target(mvs, bases, cand)
        if length(rjb["jobs"]) > 0
            ind = sortperm(-rjb["n_viable"])[1]
            job = rjb["jobs"][ind]
            b = rjb["cands"][ind]
            b_i += 1
            bases1[b_i] = b
            j_i += 1
            jobs1[j_i] = job
            println(job)
            println(b)
        end
    end
end

In [ ]:
inits = get_inits(best_cost);
targets = get_targets(scores, 80);
for i in 1:100
    targ = rand(targets)
    println(targ)
    mr = find_mods_for_target(scores, inits, targ, 1, 2, 1)
    if length(mr["cands"]) > 0
        ind = sortperm([c.cost for c in mr["cands"]])[1]
        cand = mr["cands"][ind]
        m_i += 1
        mods1[m_i] = mr["mods"][ind]
        println(mods1[m_i])
        # find base and job for cand
        rjb = find_jobs_and_bases_for_target(mvs, bases, cand)
        if length(rjb["jobs"]) > 0
            ind = sortperm(-rjb["n_viable"])[1]
            job = rjb["jobs"][ind]
            b = rjb["cands"][ind]
            b_i += 1
            bases1[b_i] = b
            j_i += 1
            jobs1[j_i] = job
            println(job)
            println(b)
        end
    end
end

In [ ]:
inits = get_inits(best_cost);
targets = get_targets(scores, 70);
for i in 1:100
    targ = rand(targets)
    println(targ)
    mr = find_mods_for_target(scores, inits, targ, 1, 1, 1)
    if length(mr["cands"]) > 0
        ind = sortperm([c.cost for c in mr["cands"]])[1]
        cand = mr["cands"][ind]
        m_i += 1
        mods1[m_i] = mr["mods"][ind]
        println(mods1[m_i])
        # find base and job for cand
        rjb = find_jobs_and_bases_for_target(mvs, bases, cand)
        if length(rjb["jobs"]) > 0
            ind = sortperm(-rjb["n_viable"])[1]
            job = rjb["jobs"][ind]
            b = rjb["cands"][ind]
            b_i += 1
            bases1[b_i] = b
            j_i += 1
            jobs1[j_i] = job
            println(job)
            println(b)
        end
    end
end

In [136]:
jobs1 = unique(jobs1[1:j_i])
mods1 = unique(mods1[1:j_i])
bases1 = bases1[1:j_i]
jobs1 = [rename_job(jobs1[i], string("j", i)) for i in 1:length(jobs1)]
mods1 = [rename_job(mods1[i], string("m", i)) for i in 1:length(mods1)];

In [208]:
@time library = get_bjms(bases1,jobs1,mods1);
length(library)

 12.478032 seconds (76.03 M allocations: 8.135 GiB, 35.21% gc time)


658892

In [139]:
serialize("temp_desLparts_parts.jex", Dict("jobs" => jobs1, "mods" => mods1, "bases" => bases1))

#### Compare "exclusive" (more efficient but less diverse) mods vs "non-exclusive" (less requirements, more costly) mods

In [142]:
length(mods1)

229

In [144]:
mods_ex = mods1[1:50]
mods_nx = mods1[171:220];


In [160]:
n_deck = 10
basesA = bases1[sortperm([rand() for i in 1:length(bases1)])[1:n_deck]]
basesB = bases1[sortperm([rand() for i in 1:length(bases1)])[1:n_deck]]
jobsA = jobs1[sortperm([rand() for i in 1:length(jobs1)])[1:n_deck]]
jobsB = jobs1[sortperm([rand() for i in 1:length(jobs1)])[1:n_deck]]
modsA = mods_ex[sortperm([rand() for i in 1:length(mods_ex)])[1:n_deck]]
modsB = mods_nx[sortperm([rand() for i in 1:length(mods_nx)])[1:n_deck]]
libA = get_bjms(basesA, jobsA, modsA)
libB = get_bjms(basesB, jobsB, modsB)
libAf = filter_nondominated(get_bjms(basesA, jobsA, modsA))
libBf = filter_nondominated(get_bjms(basesB, jobsB, modsB))
res_ffp = ffp2(libAf, libBf, 100000);

In [161]:
libAf[res_ffp.w1 .> 0.01]

3-element Vector{Spud}:
 Spud("m36 j23 b2576", 5, 3, 3, 7, 8, 8, 999, 97)
 Spud("m50 j176 b1058", 3, 5, 8, 6, 3, 4, 999, 97)
 Spud("m50 j23 b2643", 3, 2, 7, 5, 3, 9, 999, 76)

In [162]:
libBf[res_ffp.w2 .> 0.01]

2-element Vector{Spud}:
 Spud("m211 j164 b132", 7, 4, 3, 1, 3, 1, 999, 99)
 Spud("m199 j121 b2363", 5, 2, 5, 4, 7, 8, 999, 95)

In [163]:
res_ffp.v

1.4350480339523046e-5

In [221]:
n_deck = 20
basesA = bases1[sortperm([rand() for i in 1:length(bases1)])[1:n_deck]]
basesB = bases1[sortperm([rand() for i in 1:length(bases1)])[1:n_deck]]
jobsA = jobs1[sortperm([rand() for i in 1:length(jobs1)])[1:n_deck]]
jobsB = jobs1[sortperm([rand() for i in 1:length(jobs1)])[1:n_deck]]
modsA = mods_ex[sortperm([rand() for i in 1:length(mods_ex)])[1:n_deck]]
modsB = mods1[sortperm([rand() for i in 1:length(mods1)])[1:n_deck]]
libA = get_bjms(basesA, jobsA, modsA)
libB = get_bjms(basesB, jobsB, modsB)
libAf = filter_nondominated(get_bjms(basesA, jobsA, modsA))
libBf = filter_nondominated(get_bjms(basesB, jobsB, modsB))
res_ffp = ffp2(libAf, libBf, 100000);

In [222]:
libAf

98-element Vector{Spud}:
 Spud("m15 j231 b1012", 4, 5, 5, 7, 3, 4, 999, 100)
 Spud("m29 j162 b1012", 4, 4, 8, 6, 5, 3, 999, 98)
 Spud("m15 j79 b1012", 5, 4, 6, 7, 2, 3, 999, 100)
 Spud("m29 j228 b1012", 5, 3, 8, 7, 3, 3, 999, 91)
 Spud("m29 j218 b1012", 4, 3, 8, 7, 4, 4, 999, 83)
 Spud("m15 j218 b1012", 4, 4, 6, 8, 3, 4, 999, 93)
 Spud("m29 j144 b1012", 4, 3, 9, 7, 4, 3, 999, 79)
 Spud("m15 j144 b1012", 4, 4, 7, 8, 3, 3, 999, 89)
 Spud("m34 j231 b1802", 2, 4, 8, 4, 6, 7, 999, 91)
 Spud("m34 j33 b1802", 4, 3, 8, 5, 7, 6, 999, 100)
 Spud("m3 j33 b1802", 3, 4, 9, 5, 7, 5, 999, 100)
 Spud("m34 j162 b1802", 2, 4, 8, 4, 7, 6, 999, 99)
 Spud("m39 j162 b1802", 1, 5, 9, 4, 7, 5, 999, 99)
 ⋮
 Spud("m35 j218 b2339", 1, 2, 9, 7, 6, 9, 999, 92)
 Spud("m41 j218 b2339", 1, 2, 9, 7, 7, 8, 999, 90)
 Spud("m10 j218 b2339", 2, 1, 9, 7, 6, 8, 999, 93)
 Spud("m41 j229 b2339", 3, 2, 8, 6, 8, 7, 999, 99)
 Spud("m18 j229 b2339", 4, 1, 8, 7, 7, 7, 999, 97)
 Spud("m39 j231 b1827", 1, 6, 7, 1, 7, 6, 999, 91)
 Sp

In [223]:
res_ffp.v

-0.1407395949724054

In [224]:
libAf[res_ffp.w1 .> 0.01]

10-element Vector{Spud}:
 Spud("m15 j218 b671", 4, 6, 2, 7, 5, 3, 999, 93)
 Spud("m2 j225 b1616", 3, 5, 4, 2, 9, 7, 999, 90)
 Spud("m29 j228 b1732", 5, 2, 8, 4, 4, 5, 999, 91)
 Spud("m34 j225 b378", 2, 7, 7, 2, 7, 4, 999, 99)
 Spud("m15 j79 b1670", 4, 5, 5, 7, 1, 5, 999, 100)
 Spud("m2 j122 b2178", 2, 4, 5, 9, 8, 7, 999, 99)
 Spud("m35 j231 b2178", 2, 3, 5, 9, 9, 8, 999, 99)
 Spud("m15 j231 b223", 4, 7, 3, 7, 5, 2, 999, 100)
 Spud("m3 j162 b223", 4, 7, 4, 6, 6, 1, 999, 99)
 Spud("m39 j231 b1827", 1, 6, 7, 1, 7, 6, 999, 91)

In [225]:
libBf[res_ffp.w2 .> 0.01]

6-element Vector{Spud}:
 Spud("m213 j111 b404", 3, 5, 4, 8, 9, 2, 999, 93)
 Spud("m225 j130 b1333", 6, 5, 3, 4, 1, 5, 999, 87)
 Spud("m225 j237 b268", 5, 6, 2, 5, 6, 2, 999, 81)
 Spud("m225 j130 b268", 6, 6, 2, 3, 5, 2, 999, 87)
 Spud("m225 j107 b268", 5, 6, 2, 5, 7, 1, 999, 99)
 Spud("m225 j191 b1206", 3, 6, 4, 5, 7, 5, 999, 100)

In [226]:
n_deck = 20
libsAf = Array{Array{Spud}}(undef, 5)
for i in 1:length(libsAf)
    basesA = bases1[sortperm([rand() for i in 1:length(bases1)])[1:n_deck]]
    jobsA = jobs1[sortperm([rand() for i in 1:length(jobs1)])[1:n_deck]]
    modsA = mods_ex[sortperm([rand() for i in 1:length(mods_ex)])[1:n_deck]]
    libA = get_bjms(basesA, jobsA, modsA)
    libAf = filter_nondominated(get_bjms(basesA, jobsA, modsA))
    libsAf[i] = libAf
end

In [227]:
n_deck = 20
libsBf = Array{Array{Spud}}(undef, 5)
for i in 1:length(libsAf)
    basesA = bases1[sortperm([rand() for i in 1:length(bases1)])[1:n_deck]]
    jobsA = jobs1[sortperm([rand() for i in 1:length(jobs1)])[1:n_deck]]
    modsA = mods1[sortperm([rand() for i in 1:length(mods1)])[1:n_deck]]
    libA = get_bjms(basesA, jobsA, modsA)
    libAf = filter_nondominated(get_bjms(basesA, jobsA, modsA))
    libsBf[i] = libAf
end

In [228]:
# matchups
mus = Array{Float64}(undef, (length(libsAf), length(libsBf)))
for i in 1:length(libsAf)
    for j in 1:length(libsBf)
        mus[i,j] = ffp2(libsAf[i], libsBf[j], 100000).v
    end
end

In [229]:
[mean(m) for m in eachrow(mus)]

5-element Vector{Float64}:
  0.027980446952291994
 -0.13453292005443301
 -0.001246924788759124
 -0.07746521897857205
  0.11725532686882625

In [230]:
[mean(m) for m in eachcol(mus)]

5-element Vector{Float64}:
  0.10103471405609221
 -0.022676077229351825
  0.016454906919937264
 -0.03354408758443554
 -0.12927874616288804

In [232]:
mus[5,5]

-0.004350391195220595

In [233]:
libsAf[5]

88-element Vector{Spud}:
 Spud("m32 j222 b2266", 6, 1, 3, 5, 5, 8, 999, 87)
 Spud("m31 j105 b2266", 6, 2, 3, 5, 5, 7, 999, 93)
 Spud("m32 j105 b2266", 7, 1, 3, 5, 4, 8, 999, 98)
 Spud("m49 j101 b323", 5, 6, 3, 3, 7, 2, 999, 93)
 Spud("m49 j222 b323", 5, 6, 2, 4, 8, 1, 999, 77)
 Spud("m1 j105 b323", 6, 6, 2, 3, 7, 1, 999, 100)
 Spud("m49 j232 b323", 5, 7, 3, 2, 8, 1, 999, 96)
 Spud("m43 j223 b323", 7, 6, 2, 3, 6, 1, 999, 93)
 Spud("m49 j223 b323", 6, 7, 2, 2, 6, 1, 999, 97)
 Spud("m49 j178 b323", 5, 6, 4, 3, 6, 1, 999, 100)
 Spud("m15 j222 b1253", 4, 3, 4, 9, 9, 3, 999, 86)
 Spud("m15 j105 b1253", 5, 3, 4, 9, 8, 3, 999, 97)
 Spud("m15 j215 b1253", 4, 4, 4, 9, 7, 4, 999, 96)
 ⋮
 Spud("m20 j105 b1817", 9, 2, 1, 9, 7, 5, 999, 95)
 Spud("m11 j105 b1817", 9, 2, 1, 9, 6, 6, 999, 99)
 Spud("m48 j232 b1817", 8, 2, 3, 9, 7, 6, 999, 93)
 Spud("m1 j232 b1817", 8, 3, 2, 8, 7, 5, 999, 88)
 Spud("m20 j223 b1817", 9, 3, 1, 9, 6, 5, 999, 84)
 Spud("m11 j223 b1817", 9, 3, 1, 9, 5, 6, 999, 88)
 Spud("m29

In [234]:
libsBf[5]

146-element Vector{Spud}:
 Spud("m64 j62 b1908", 2, 2, 7, 6, 9, 8, 999, 97)
 Spud("m64 j223 b1908", 2, 3, 7, 4, 9, 6, 999, 91)
 Spud("m123 j140 b1908", 1, 3, 9, 7, 8, 5, 999, 99)
 Spud("m123 j109 b1908", 2, 3, 7, 7, 9, 5, 999, 97)
 Spud("m112 j140 b1486", 2, 3, 9, 9, 4, 5, 999, 97)
 Spud("m136 j175 b1486", 1, 5, 7, 8, 5, 5, 999, 95)
 Spud("m112 j197 b1486", 2, 3, 8, 9, 5, 5, 999, 96)
 Spud("m123 j124 b1486", 2, 4, 8, 8, 5, 4, 999, 97)
 Spud("m112 j124 b1486", 3, 3, 8, 7, 5, 5, 999, 95)
 Spud("m28 j234 b2177", 3, 2, 8, 8, 8, 6, 999, 95)
 Spud("m123 j234 b2177", 3, 2, 7, 9, 8, 6, 999, 93)
 Spud("m64 j62 b2177", 2, 1, 7, 9, 9, 9, 999, 97)
 Spud("m227 j223 b2177", 2, 3, 7, 8, 8, 6, 999, 97)
 ⋮
 Spud("m43 j235 b1333", 7, 4, 3, 5, 2, 5, 999, 98)
 Spud("m123 j140 b1333", 5, 5, 5, 7, 1, 4, 999, 99)
 Spud("m98 j70 b1333", 5, 6, 3, 4, 1, 6, 999, 95)
 Spud("m136 j222 b1333", 5, 5, 3, 6, 3, 5, 999, 90)
 Spud("m43 j239 b1333", 7, 4, 3, 6, 2, 4, 999, 83)
 Spud("m160 j136 b1333", 5, 6, 3, 5, 2, 4, 99

In [235]:
rf = ffp2(libsAf[5], libsBf[5], 100000);

In [244]:
df = spuds_to_df(libsAf[5])
df[:, :c0] = [scores[a.h,a.f,a.l,a.p,a.r,a.s] for a in libsAf[5]]
df[:, :w] = rf.w1
println(df[sortperm(-rf.w1)[1:sum(rf.w1 .> .01)],:])

17×11 DataFrame
 Row │ name            h      f      l      p      r      s      a1     cost   c0     w         
     │ String          Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64   
─────┼──────────────────────────────────────────────────────────────────────────────────────────
   1 │ m48 j232 b1497      7      2      4      2      7      5    999     93     63  0.143664
   2 │ m33 j151 b1431      3      6      5      3      4      7    999     84     76  0.133582
   3 │ m43 j223 b1162      7      5      1      6      5      3    999     93     76  0.127698
   4 │ m24 j178 b2171      5      1      7      5      8      6    999     93     83  0.121423
   5 │ m43 j223 b323       7      6      2      3      6      1    999     93     85  0.110493
   6 │ m33 j232 b2419      2      7      3      1      4      9    999     93     73  0.073845
   7 │ m32 j105 b2171      6      1      4      6      9      7    999     98     60  0.0523639
   8 │ m29 j80 b1168       

In [245]:
df = spuds_to_df(libsBf[5])
df[:, :c0] = [scores[a.h,a.f,a.l,a.p,a.r,a.s] for a in libsBf[5]]
df[:, :w] = rf.w2
println(df[sortperm(-rf.w2)[1:sum(rf.w2 .> .01)],:])

17×11 DataFrame
 Row │ name             h      f      l      p      r      s      a1     cost   c0     w         
     │ String           Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64   
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │ m43 j234 b1012       7      3      5      7      2      3    999    100     81  0.17818
   2 │ m160 j223 b1333      6      6      3      4      1      4    999     97     83  0.168454
   3 │ m160 j223 b1206      4      6      4      5      6      3    999     97     69  0.114872
   4 │ m123 j197 b1206      3      5      5      8      7      3    999     98     77  0.0817407
   5 │ m160 j223 b1735      4      6      5      4      3      5    999     97     71  0.069189
   6 │ m160 j136 b1509      3      6      3      6      6      4    999     90     78  0.0552993
   7 │ m136 j175 b1058      1      7      7      6      4      4    999     95     68  0.0544705
   8 │ m136 j239

#### Optimize slackness

In [246]:
function generate_parts_targeted(mvs,bases,best_cost,scores,target_cost,dist_thres,n_reqs,slack,n_parts)
    bases1 = Array{SpudBase}(undef, 10000)
    jobs1 = Array{SpudMod}(undef, 10000)
    mods1 = Array{SpudMod}(undef, 10000)
    b_i = 0
    j_i = 0
    m_i = 0
    inits = get_inits(best_cost);
    targets = get_targets(scores, target_cost);
    while b_i < n_parts
        targ = rand(targets)
        #println(targ)
        mr = find_mods_for_target(scores, inits, targ, dist_thres, n_reqs, slack)
        if length(mr["cands"]) > 0
            ind = sortperm([c.cost for c in mr["cands"]])[1]
            cand = mr["cands"][ind]
            m_i += 1
            mods1[m_i] = mr["mods"][ind]
            #println(mods1[m_i])
            # find base and job for cand
            rjb = find_jobs_and_bases_for_target(mvs, bases, cand)
            if length(rjb["jobs"]) > 0
                ind = sortperm(-rjb["n_viable"])[1]
                job = rjb["jobs"][ind]
                b = rjb["cands"][ind]
                b_i += 1
                bases1[b_i] = b
                j_i += 1
                jobs1[j_i] = job
                #println(job)
                #println(b)
            end
        end
    end
    jobs1 = unique(jobs1[1:j_i])
    mods1 = unique(mods1[1:j_i])
    bases1 = bases1[1:j_i]
    jobs1 = [rename_job(jobs1[i], string("j", i)) for i in 1:length(jobs1)]
    mods1 = [rename_job(mods1[i], string("m", i)) for i in 1:length(mods1)];
    return Dict("bases"=>bases1, "jobs"=>jobs1, "mods"=>mods1)
end



generate_parts_targeted (generic function with 1 method)

In [ ]:
vs = Array{Float64}(undef, 10);
results = Array{Dict}(undef, length(vs));
for i in 1:length(vs)
    partsA = generate_parts_targeted(mvs,bases,best_cost,scores,70,2,2,1, 10);
    libAf = filter_nondominated_recursive(get_bjms(partsA["bases"], partsA["jobs"], partsA["mods"]));
    partsB = generate_parts_targeted(mvs,bases,best_cost,scores,70,2,2,2, 10);
    libBf = filter_nondominated_recursive(get_bjms(partsB["bases"], partsB["jobs"], partsB["mods"]));
    rf = ffp2(libAf, libBf, 100000);
    vs[i] = rf.v;
    results[i] = Dict("partsA" => partsA, "partsB" => partsB, "libAf" => libAf, "libBf" => libBf, "rf" => rf)
end